# NET2008 - DevOps - Fall 2020 - Assignment 5

Use Ansible's Python API to translate the steps in Lab 09 & 10 into Python code, achieving the same results. Specifically, use Ansible's Python API to complete the following:

1. Load inventory using Python code, print the names, IP addresss and group names of all hosts. Ping all these hosts and output the results. (10%)
1. Run the Wildflyapp playbook using Python code to install the app on the group `Wildflyservers`. Use the existing playbook file from the lab and pass is to Python code for running the playbook. Print the results. Read and verify the response of the wildfly servers from your python code. (30%)
1. Run the load balancer playbook using Python code to install the load balancer on the group `lbservers`. Use the existing playbook file from the lab and pass is to Python code for running the playbook. Print the results. Read and verify the response of the wildfly servers from your python code. (30%)
1. Run the steps of Section _"Handling Secrets with Ansible Vault"_ in the lab. Use the existing playbook file from the lab and pass is to Python code for running the playbook. Print the results. Read and verify the response of the wildfly servers from your python code. (30%)

## References

* [Using Runner as a Python Module Interface to Ansible](https://ansible-runner.readthedocs.io/en/latest/python_interface.html)

## Python Library Installation

In [ ]:
!pip3 install ansible_runner

# If you don't have pip3, uncomment the line below to use pip instead
# !pip install ansible_runner

## Answer

In [ ]:
import ansible_runner
import os
import subprocess

# Write your answer here by modifying or extending the sample code below

#Making a class to store basic host information
class host_info:
    def __init__(self, hostname):
        self.hostname = hostname
        self.groups = []
    def setIP(self, ip_address):
        self.ip_address = ip_address
    def addGroup(self, group_name):
        self.groups.append(group_name)

#Creating some variables used to obtain group name information, and give it to the correct hosts

#List to contain host_info objects
hosts  = []    

#Function to store the data in the host_info list
def get_host_info(file):
    #Loop to read every line in the file
    for line in host_file:
        #If the line is empty, go to the next loop iteration
        if line[0] == '\n':
            continue
        #Setting the current group variable, used to add information to the hosts
        if ('[' in line) and (']' in line):
            current_group = line[line.index('[')+ 1 : line.index(']')]
            continue
        #If statement designed to only find the first line containing a hostname (Which only happens in lines with a space)
        if (' ' in line) and ("ansible_ssh" not in line):
            #Create a new object, and set the hostname, IP, as well as add the current group
            hosts.append(host_info(line[:line.index(' ')]))
            hosts[-1].setIP(line[line.index('=') + 1 : -1])
            hosts[-1].addGroup(current_group)
            continue
        #If the current hostname hasn't been set as a part of the current group yet (but needs to), add the group
        for i in hosts:
            #Used to find the correct host to add the group to
            if i.hostname == line[:-1]:
                #Finally, add the group to the hostname
                i.addGroup(current_group)
    #Finally, the function printrs out the information
    print("The following host information has been found:")
    print("_______________________________________________________")
    print("Hostname \t | IP Address \t | Group(s)")
    print("-------------------------------------------------------")
    for i in hosts:
        print("{:<18}{:<16}{}".format(i.hostname, i.ip_address, ", ".join(i.groups)))

#Steps to read the content of the hosts file
try:
    host_file = open("inventory/hosts", "r")
except:
    print("Couldn't find the hosts file, or the file does not exist")
else:
    get_host_info(host_file)
    host_file.close()

#Ansible portion of the code, running through the different playbooks 
    
print("_______________________________________________________")
print("                 Running ping.yml")
print("-------------------------------------------------------\n")

ping_runner = ansible_runner.interface.run(private_data_dir=".", playbook="ping.yml")

print("_______________________________________________________")
print("                 Running site.yml")
print("-------------------------------------------------------\n")

site_runner = ansible_runner.interface.run(private_data_dir=".", playbook="site.yml")

print("_______________________________________________________")
print("                 Testing site.yml")
print("-------------------------------------------------------\n")

#Running the curl command as a test
#The loop is to run it on every host
for i in hosts:
    #If the host isn't a wildflyserver, skip it
    if "wildflyservers" not in i.groups:
        continue
    #os.system("curl -w '\n' http://{}:8080".format(i.ip_address))
    output = subprocess.check_output(["curl", "-w", "'\n'", "http://{}:8080".format(i.ip_address)])
    print(output)

print("_______________________________________________________")
print("                  Running lb.yml")
print("-------------------------------------------------------\n")

loadbalancer_runner = ansible_runner.interface.run(private_data_dir=".", playbook="lb.yml")

print("_______________________________________________________")
print("                  Testing lb.yml")
print("-------------------------------------------------------\n")

#Loop to go through the hosts and find any lbservers
for i in hosts:
    #If the host isn't a wildflyserver, skip it
    if "lbservers" not in i.groups:
        continue
    #os.system("curl -w '\n' http://{}:8080".format(i.ip_address))
    #Loop to run the test a few times
    for j in range(4):
        #To make sure only lbservers are tested
        if "lbservers" not in i.groups:
            continue 
        output = subprocess.check_output(["curl", "-w", "'\n'", "http://{}".format(i.ip_address)])
        print(output)

print("_______________________________________________________")
print("              Running site.yml with vault")
print("-------------------------------------------------------\n")

print("The first site.yml run also used the vault, but I included this to include every step in the lab instructions\n")

vault_runner = ansible_runner.interface.run(private_data_dir=".", playbook="site.yml")
        

print("_______________________________________________________")
print("                   Testing vault")
print("-------------------------------------------------------\n")

print("All of the playbooks in assignment 5 were run using the vault and password")
print("In Assignment5/env/cmdline, I included the parameter '--vault-password-file mypassword'")
print("Without the 'Assignment5/env/cmdline' file, every playbook in this assignment would return an error saying they couldn't be decrypted\n")

try:
    command_file = open("env/cmdline", "r")
except:
    print("The cmdline file does not exist, or could not be found")
else:
    print("The contents of the cmdline file are:")
    for command_line in command_file:
        print(command_line)
    command_file.close()
    print()

try:
    vault_file = open("project/group_vars/dev/wildflyservers/vault.yml", "r")
except:
    print("The vault file does not exist, or could not be found")
else:
    print("The contents of the vault.yml file are:")
    for vault_line in vault_file:
        print(vault_line)
    vault_file.close()